# Synthetic Tabular Data Generation by NeMo GPT

Big data, new algorithms and fast computers are the 3 main factors that make the modern AI revolution possible. Data poses a big challenge for enterprises for various reasons: difficulty  of data labeling, strong data governance, limited data availability etc. Synthetic data generation is proposed as a solution to the data problem as it directly addresses the data challenges. Two popular generative models like  Variational Auto-Encoder [VAE](https://en.wikipedia.org/wiki/Variational_autoencoder) and Generative Adversarial Network [GAN](https://en.wikipedia.org/wiki/Generative_adversarial_network) models have achieved some success in the past. However, a good generative model should generate the data following the same distribution as the training data. There are some known flaws with the VAE and GAN models for synthetic data generation. Most prominently, the [mode collapse problem](https://developers.google.com/machine-learning/gan/problems) in the GAN model causes the generated data to miss some modes in the training data distribution. While the VAE has the difficulty of generating sharp data points due to the non-autoregressive loss.

Recently, [Transformer models](https://arxiv.org/abs/1706.03762) achieved huge success in the natural language processing domain.  The self-attention encoding and decoding architecture of the transformer model is proven to be accurate in modeling the data distribution and scalable to large datasets. [OpenAI’s GPT3](https://openai.com/blog/gpt-3-apps/) uses the decoder part of the transformer model and has 175B parameters. It has been widely used across varying categories and industries, from productivity and education to creativity and games. GPT3 turns out to be a superior generative model. In this tutorial, we are interested in exploring the ideas of applying the GPT model for synthetic data generation. 

Unlike typical neural networks, GPT models are usually large in size. It is challenging to fit a large GPT model into a single GPU. Luckily, [NeMo](https://github.com/NVIDIA/NeMo) is an open source tool that is capable of efficiently training very large (hundreds of billions of parameters) language models with both model and data parallelism.  It is easy to use too. In the following sections, we will show how to train a GPT model step by step to generate synthetic credit card data.


### Get the Credit Card Transaction Data 

The synthetic credit card transaction dataset is provided at [IBM TabFormer Github repo](https://github.com/IBM/TabFormer). Go ahead to download it at this [direct link](https://ibm.box.com/v/tabformer-data). Put the downloaded and decompressed `card_transaction.v1.csv` file at the same directory as this this notebook.

### Data Cleaning and Formatting

First let's import the necessary Python libraries for the tutorial.

In [1]:
import pandas as pd
from nemo.collections.common.tokenizers.column_coder import ColumnCodes
from omegaconf import OmegaConf
import pickle
from pandas.api.types import is_string_dtype
from nemo.collections.common.tokenizers.tabular_tokenizer import TabularTokenizer
import json
import wget
import requests
import json
import functools
from multiprocessing import Pool

In [2]:
BRANCH = 'r1.23.0'
DATA_PATH='.'
TRANSACTIONS=DATA_PATH+'/card_transaction.v1.csv'
#CHECKPOINTS='/chk_points'
CHECKPOINTS=DATA_PATH+'/checkpoints'
CC_OUTPUT_P=DATA_PATH+'/credit_card_coder.pickle'
CC_OUTPUT_J=DATA_PATH+'/credit_card.jn'
print (TRANSACTIONS, CHECKPOINTS, CC_OUTPUT_P, CC_OUTPUT_J)

./card_transaction.v1.csv ./checkpoints ./credit_card_coder.pickle ./credit_card.jn


After decompressing the tar file,  we load the dataset into pandas dataframe and examine the top few rows.

In [4]:
df = pd.read_csv(TRANSACTIONS)
df.head()

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
0,0,0,2002,9,1,06:21,$134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,NaN,No
1,0,0,2002,9,1,06:42,$38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
2,0,0,2002,9,2,06:22,$120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
3,0,0,2002,9,2,17:45,$128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754.0,5651,NaN,No
4,0,0,2002,9,3,06:23,$104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750.0,5912,NaN,No


In the following data cleaning step, we fix the `Amount` column that is not a number, convert the `Time` column into `hour` and `minutes` and replace the `,` character with space so we can use `,` as delimiter.

In [5]:
# remove the dollar string and convert amount to float
df['Amount'] = df['Amount'].str.replace('$', '').astype('float')
# fix the zip column
df['Zip'] = pd.array([None if pd.isna(x) else int(x) for x in df['Zip']])

# parse the time into minute and second
df['Hour'] = df['Time'].str[0:2]
df['Minute'] = df['Time'].str[-2:]
# remove the 'Time' Column once it is parsed
del df['Time']
# all the columns used for later processing 
all_columns = ['User', 'Year', 'Month', 'Day', 'Hour', 'Minute', 'Card', 'Amount', 'Use Chip',
           'Merchant Name', 'Merchant City', 'Merchant State', 'Zip', 'MCC',
           'Errors?', 'Is Fraud?']
# these are the columns used to train the model
columns = ['Year', 'Month', 'Day', 'Hour', 'Minute', 'Card', 'Amount', 'Use Chip',
           'Merchant Name', 'Merchant City', 'Merchant State', 'Zip', 'MCC',
           'Errors?', 'Is Fraud?']

float_columns = ['Amount']
category_columns = ['Year', 'Month', 'Day', 'Hour', 'Minute', 'Card', 'Use Chip',
                    'Merchant Name', 'Merchant City', 'Merchant State', 'MCC',
                    'Errors?', 'Is Fraud?']
integer_columns = ['Zip']

for col in category_columns:
    if is_string_dtype(df[col].dtype):
        df[col] = df[col].str.replace(',', ' ')

# after preprocessing
df = df[all_columns]
df.head()

[NeMo W 2024-02-26 22:41:54 nemo_logging:349] /tmp/ipykernel_45614/528401009.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
      df['Amount'] = df['Amount'].str.replace('$', '').astype('float')
    


,User,Year,Month,Day,Hour,Minute,Card,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
0,0,2002,9,1,06,21,0,134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750,5300,NaN,No
1,0,2002,9,1,06,42,0,38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754,5411,NaN,No
2,0,2002,9,2,06,22,0,120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754,5411,NaN,No
3,0,2002,9,2,17,45,0,128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754,5651,NaN,No
4,0,2002,9,3,06,23,0,104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750,5912,NaN,No


There are 24M records with 12 fields. We need to convert the structured tabular data into a format that NeMo GPT can consume. The NeMo GPT uses training data in a loose json format, with one json containing a text sample per line. For example:
```json
{"text": "The quick brown fox"}
{"text": "jumps over the lazy dog"}
```
We can concat each fields of the transactions together like:
```json
{"text": "0,0,2002,9,1,06:21,$134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,NaN,No\n0,0,2002,9,1,06:42,$38.48,Swipe,Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No..."}
...
```
In this way, the tabular data is converted into NLP text corpus. 

### NeMo GPT Workflow
```
                       +-----------+
      +--------------->|Tokenizer  |
      |                +-----+-----+
      |                      |
+-----+-----+          +-----v--------+      +---------+
|  Raw Data +--------->| Preprocess   +----->|Pretrain |
|           |          |              |      +-----+---+
+-----------+          +--------------+            |
                                                   |
                                           +-------v----------+
                                           |  Inference       |
                                           | (Text Generation)|
                                           +------------------+
```
As shown in the figure above, it illustrates the steps we are going to take in this tutorial. Having the raw text data ready, we first train a tokenizer to efficiently tokenize the text.  As NeMo only cares about sequences of tokens, a preprocess step is used to convert the raw text data into sequences of token index. It results in one binary file and an index file so data loading in the training step is more efficient.  The next step is the most time consuming pre-training step in which sequences of tokens are fed into the NeMo GPT to predict the next tokens. The prediction error is used as the loss to train the GPT weights. After training the GPT model, it can be put into an inference mode to generate synthetic data following the same distribution as the training dataset. 

### Tokenizer training
It is natural to select the generic GPT BPE tokenizer to convert the text into tokens. However, there are a few problems of using this approach. When GPT BPE tokenizer splits the text into tokens, the number of tokens are usually not fixed for the same column at different rows, because the number is determined by the occurrence frequencies of the individual sub-tokens. This means the structure information is lost if we use the NLP tokenizer. Another problem with the NLP tokenizer is the long string consists of a large number of tokens, which is wasteful considering the NeMo GPT has limited capacity of modeling the sequences of tokens. For example the "Merchant Name" "3527213246127876953" need at least 7 tokens to code it ([35][27][213][246][127][876][953]). 

As shown in the [TabFormer paper](https://arxiv.org/abs/2011.01843), a good solution is to build a specialized tokenizer for the tabular data that considers the table structural information. The TabFormer uses a single token for each of the columns which can cause either accuracy loss if the number of tokens is small for the column, or weak generalization if the number of tokens is too large. We improve it by using multiple tokens to code the columns. For example, the floating number "134.09" can be tokenized into multiple integers.

The following code trains a special encoder/decoder used in the tokenizer. It uses `float` for the `Amount` column, `int` for the `zip` column and `category` for the rest of the columns. The trained encoder and decoder are saved into a file that can be loaded later. We choose to use 3 tokens to encode the floating and int numbers in this example.

In [6]:

tab_structure = []
for c in columns:
    if c in float_columns:
        item = {
            "name": c,
            "code_type": "float",
            "args": {
                "code_len": 3,  # number of tokens used to code the column
                "base": 32,   # the positional base number. ie. it uses 32 tokens for one digit
                "fillall": True, # whether to use full base number for each token or derive it from the data.
                "hasnan": False, # can it handles nan or not
                "transform": "yeo-johnson" # can be ['yeo-johnson', 'quantile', 'robust'], check https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing 
            }
        }
    elif c in integer_columns:
        item = {
            "name": c,
            "code_type": "int",
            "args": {
                "code_len": 3,  # number of tokens used to code the column
                "base": 47,   # the positional base number. ie. it uses 32 tokens for one digit
                "fillall": True, # whether to use full base number for each token or derive it from the data.
                "hasnan": True, # can it handles nan or not
            }
        }
    else:
        item = {
            "name": c,
            "code_type": "category",
        }
    tab_structure.append(item)
print(OmegaConf.to_yaml(tab_structure))
print(columns)

example_arrays = {}
for col in tab_structure:
    col_name = col['name']
    if col_name in category_columns:
        example_arrays[col_name] = [i.strip() for i in df[col_name].astype(str).unique()]
    else:
        example_arrays[col_name] = df[col_name].dropna().unique()
cc = ColumnCodes.get_column_codes(tab_structure, example_arrays)
print('each row uses', sum(cc.sizes)+ 1, 'tokens')
with open(CC_OUTPUT_P, 'wb') as handle:
    pickle.dump(cc, handle)


- name: Year
  code_type: category
- name: Month
  code_type: category
- name: Day
  code_type: category
- name: Hour
  code_type: category
- name: Minute
  code_type: category
- name: Card
  code_type: category
- name: Amount
  code_type: float
  args:
    code_len: 3
    base: 32
    fillall: true
    hasnan: false
    transform: yeo-johnson
- name: Use Chip
  code_type: category
- name: Merchant Name
  code_type: category
- name: Merchant City
  code_type: category
- name: Merchant State
  code_type: category
- name: Zip
  code_type: int
  args:
    code_len: 3
    base: 47
    fillall: true
    hasnan: true
- name: MCC
  code_type: category
- name: Errors?
  code_type: category
- name: Is Fraud?
  code_type: category

['Year', 'Month', 'Day', 'Hour', 'Minute', 'Card', 'Amount', 'Use Chip', 'Merchant Name', 'Merchant City', 'Merchant State', 'Zip', 'MCC', 'Errors?', 'Is Fraud?']
each row uses 20 tokens


Let's give it a try to play with encoder and decoder for "Amount" and "Merchant City" columns

In [7]:
float_str = '134.09'
token_ids = cc.encode('Amount', float_str)
print('token ids for {} is {}'.format(float_str, token_ids))
amt_str = cc.decode('Amount', token_ids)
print('recovered Amt for {} is {}'.format(float_str, amt_str))

city_str = 'Monterey Park'
token_ids = cc.encode('Merchant City', city_str)
print('token ids for {} is {}'.format(city_str, token_ids))
amt_str = cc.decode('Merchant City', token_ids)
print('recovered City for {} is {}'.format(city_str, amt_str))

token ids for 134.09 is [234, 225, 172]
recovered Amt for 134.09 is 133.9723
token ids for Monterey Park is [108435]
recovered City for Monterey Park is Monterey Park


Using 3 tokens for the float column `Amount`, this is a tiny loss of accuracy. If better accuracy is needed, more tokens can be used to train the tokenizer. For the categorical column "Merchant City", the string "Monterey Park" only needs one token to encode it. Once we have the encoder and decoder ready, the tokenizer is easy to implement. We can consider the tabular data structure information to encode each of the columns. The decoding simply counts the number of tokens to infer the tabular structure since the number of the tokens are fixed for each of the columns.

There is one more thing that we need to take special care of before generating the loose json file for NeMo. `<|endoftext|>` is a special token that NeMo recognizes to indicate the beginning and end of the document. The attention mask will stop at the boundary of `<|endoftext|>` token so no attention can be applied across it. To model the temporal information in the time series, we want to make sure the `<|endoftext|>` is added between the continuous sections. For example, in this credit card dataset, there are 2000 users. Each user's transactions are one long time series sequence. The `<|endoftext|>` is added at the end of the long sequences. In this way, NeMo applies attention to learn the temporal correlation in the transactions for the user but not across users. 

We have provided the Python code to convert the CSV file into the loose json format.

In [8]:
delimiter = ','
eod_str = '<|endoftext|>'
int_nan = '<NA>'


def get_docs_from_df(one_group):
    user_df = one_group[1]
    total = []
    # sort the rows by time, so the model can learn temporal info
    sub_df = user_df.sort_values(['Year', 'Month', 'Day', 'Hour', 'Minute'])[columns]
    start_rows = range(0, doc_augements)
    full_msgs = []
    for start in start_rows:
        full_msgs = []
        doc_df = sub_df.iloc[start:]
        count = 0
        df_size = len(doc_df)       
        for row in doc_df.iterrows():
            count += 1
            items = row[1].values
            str_items = [str(items[i]).replace(int_nan, 'nan') for i in range(len(items))]
            if count == df_size:
                # append eod at the end of the doc
                full_msgs.append(delimiter.join(str_items)+eod_str)
            else:
                full_msgs.append(delimiter.join(str_items))
        # use end of line to separate rows
        text = '\n'.join(full_msgs)
        text_doc = {'text': text}
        doc = json.dumps(text_doc)+'\n'
        total.append(doc)
    return total


def gen_one_doc(user_group, n_cores):
    udfs = list(user_group)
    pool = Pool(n_cores)
    docs = pool.map(get_docs_from_df, udfs)
    pool.close()
    pool.join()
    return functools.reduce(lambda a, b: a + b, docs)

# number of document augmentation
doc_augements = 2

user_group = df.groupby('User')

with open(CC_OUTPUT_J, 'w') as f:
    docs = gen_one_doc(user_group, 30)
    for doc in docs:
        f.write(doc)

After running the above cell, it converts the CSV file to the loose json format that the NeMo Megatron can use. It generates one document for each user. Each document ends with the `<|endoftext|>` token. We augment the data by shifting it by one transaction so the relative position of the input credit card transaction data is slightly different. 

Having the loose JSON file ready, we can test the customized Tabular Tokenizer.

In [9]:
tokenizer = TabularTokenizer(CC_OUTPUT_P, delimiter=',')

with open(CC_OUTPUT_J, 'r') as f:
    for line in f:
        break

text = json.loads(line)['text']
r = tokenizer.text_to_tokens(text)
ids = tokenizer.tokens_to_ids(r)
tex = tokenizer.ids_to_tokens(ids)
decoded_text = tokenizer.ids_to_text(ids)
show_num_lines = 5
print('raw input text\n', '\n'.join(text.split('\n')[:show_num_lines]))
print('tokens\n', r[:show_num_lines*len(tokenizer.code_column.columns)])
print('token ids\n', ids[:show_num_lines*(sum(tokenizer.code_column.sizes)+1)])
print('decoded tokens\n', tex[:show_num_lines*(sum(tokenizer.code_column.sizes)+1)])
print('decoded text\n', '\n'.join(decoded_text.split('\n')[:show_num_lines]))

raw input text
 2002,9,1,06,21,0,134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750,5300,nan,No
2002,9,1,06,42,0,38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754,5411,nan,No
2002,9,2,06,22,0,120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754,5411,nan,No
2002,9,2,17,45,0,128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754,5651,nan,No
2002,9,3,06,23,0,104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750,5912,nan,No
tokens
 ['2002', '9', '1', '06', '21', '0', '134.09', 'Swipe Transaction', '3527213246127876953', 'La Verne', 'CA', '91750', '5300', 'nan', 'No', '\n', '2002', '9', '1', '06', '42', '0', '38.48', 'Swipe Transaction', '-727612092139916043', 'Monterey Park', 'CA', '91754', '5411', 'nan', 'No', '\n', '2002', '9', '2', '06', '22', '0', '120.34', 'Swipe Transaction', '-727612092139916043', 'Monterey Park', 'CA', '91754', '5411', 'nan', 'No', '\n', '2002', '9', '2', '17', '45', '0', '128.95', 'Swipe Transaction

The TabularTokenizer understands the Table structure so it can convert back and forth between the tabular data text and the token ids.


### Model configuration


We define the following constants to configure the GPT model 

In [10]:
NUM_LAYERS = 4
NUM_GPUS = 1
HIDDEN_SIZE = 1024
NUM_ATTENTION_HEADS = 16
SEQ_LENGTH = 1024
TENSOR_MP_SIZE = 1
PIPELINE_MP_SIZE = 1

### Preprocess

The loose json above is processed into a binary format for training. To convert the json into mmap, cached index file, we use the `preprocess_data_for_megatron.py` script to prepare the data.


Download the python file and run the preprocess. Note we use `30` CPU workers for the preprocessing step. Please adjust it according to your compute environment.

In [11]:
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/scripts/nlp_language_modeling/preprocess_data_for_megatron.py')

'preprocess_data_for_megatron.py'

In [12]:
!python preprocess_data_for_megatron.py \
    --input={CC_OUTPUT_J} \
    --json-keys=text \
    --tokenizer-library=tabular \
    --vocab-file={CC_OUTPUT_P} \
    --tokenizer-type=Tabular \
    --output-prefix=tabular_data \
    --delimiter=, \
    --workers=30


[NeMo W 2024-02-26 22:46:30 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
      warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
    
Vocab size: 114540
Output prefix: tabular_data
Time to startup: 0.032488107681274414
Processing file ./credit_card.jn 1/1
Processed 100 documents (1.021828932839258 docs/s, 1.223987290363381 MB/s).
Processed 200 documents (2.0070984452758847 docs/s, 2.2554847736165065 MB/s).
Processed 300 documents (2.9509047117804736 docs/s, 3.2471806843266005 MB/s).
Processed 400 documents (3.8698379319961216 docs/s, 4.170088455163126 MB/s).
Processed 500 documents (4.7620744398028725 docs/s, 5.050596277956267 MB/s).
Processed 600 documents (5.610316443121176 docs/s, 5.966976852574457 MB/s).
Processed 700 documents (6.425460216602764 docs/s, 6.957510225382692 MB/s).
Processed 800 documents (5.412526937616

In [13]:
!ls $DATA_PATH/tabular*.*

./tabular_data_text_document.bin  ./tabular_data_text_document.idx


After running the script, two new files are generated in the current directory: `tabular_data_text_document.bin` and `tabular_data_text_document.idx`. The `bin` file is the `mmap` binary file and the `idx` is the index file to look up the sentences in the binary file. They will be used for the following pre-training step.

### Pretrain

We are ready to pretrain the NeMo GPT model. As large models can be quite difficult to train due to memory constraints, NeMo makes it possible by using both Tensor model parallel and Pipeline model parallelism that enables training transformer models with billions of parameters. On top of the model parallelism, we can apply the popular data parallelism to the training to fully utilize all the GPUs in the cluster. In this [paper](https://arxiv.org/pdf/2104.04473.pdf), it provides a few tips about how to setup the model parallelism and data parallelism optimally:

1. When considering different forms of model parallelism, tensor model parallelism should generally be used up to degree 𝑔 when using 𝑔-GPU servers, and then pipeline model parallelism can be used to scale up to larger models across server.
2. When using data and model parallelism, a total model-parallel size of 𝑀 = 𝑡 · 𝑝 should be used so that the model’s parameters and intermediate metadata fit in GPU memory; data parallelism can be used to scale up training to more GPUs.
3. The optimal microbatch size 𝑏 depends on the throughput and memory footprint characteristics of the model, as well as the pipeline depth 𝑝, data-parallel size 𝑑, and batch size 𝐵.

In this tutorial, we only concern with training a model that fits into a single GPU. we set the tensor model parallel and pipeline model parallelism parameter to 1. Feel free to adjust it to train a larger model according to the available compute resources. As we know from OpenAI's [scaling law of natural language](https://arxiv.org/abs/2001.08361), the larger the model, the better the performance is.

Download and run the pretraining task script below. Note, this is the most time consuming step. It takes days for the task to converge depending on the computation environment, model size.

In [14]:
!mkdir -p conf
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/language_modeling/megatron_gpt_pretraining.py')
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/language_modeling/conf/megatron_gpt_config.yaml', out='conf')

'conf/megatron_gpt_config.yaml'

In [15]:
    !python megatron_gpt_pretraining.py \
                trainer.devices={NUM_GPUS} \
                trainer.accelerator=gpu \
                trainer.log_every_n_steps=100 \
                trainer.val_check_interval=500 \
                trainer.accumulate_grad_batches=1 \
                trainer.max_steps=10000 \
                trainer.precision=16 \
                trainer.gradient_clip_val=1.0 \
                exp_manager.exp_dir=gpt_creditcard_results \
                model.tensor_model_parallel_size={TENSOR_MP_SIZE} \
                model.pipeline_model_parallel_size={PIPELINE_MP_SIZE} \
                model.optim.name=fused_adam \
                model.optim.lr=2e-4 \
                model.optim.sched.warmup_steps=2 \
                model.optim.sched.constant_steps=2 \
                model.optim.sched.min_lr=8e-5 \
                model.max_position_embeddings={SEQ_LENGTH} \
                model.encoder_seq_length={SEQ_LENGTH} \
                model.data.seq_length={SEQ_LENGTH} \
                model.tokenizer.type=Tabular \
                model.tokenizer.library=tabular \
                model.tokenizer.vocab_file={CC_OUTPUT_P} \
                model.tokenizer.delimiter=\',\' \
                model.data.eod_mask_loss=True \
                model.data.splits_string=\'3800,198,2\' \
                model.num_layers={NUM_LAYERS} \
                model.hidden_size={HIDDEN_SIZE} \
                model.num_attention_heads={NUM_ATTENTION_HEADS} \
                model.activations_checkpoint_method='block' \
                model.activations_checkpoint_num_layers=1 \
                model.data.data_prefix=[tabular_data_text_document]

[NeMo W 2024-02-26 22:54:24 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
      warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
    
[NeMo W 2024-02-26 22:54:25 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
    See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
      ret = run_job(
    
[NeMo I 2024-02-26 22:54:25 megatron_gpt_pretraining:34] 
    
    ************** Experiment configuration ***********
[NeMo I 2024-02-26 22:54:25 megatron_gpt_pretraining:35] 
    name: megatron_gpt
    restore_from_path: null
    trainer:
      devices: 1
      num_nodes: 1
      accelerator: gpu
      precision: 16
      logger: false
      enabl

The pretraining will save the checkpoint files periodically at the location specified by `exp_manager.exp_dir`. In this case, it creates a directory called `gpt_creditcard_results`. The number `trainer.val_check_interval` controls the frequency of validation evaluation. We use 500 in this example. The trainer will preserve the checkpoint files which have the top `k` best validation error, by default, `k=10`. 
While the training job is running, we can use the `tensorboard` to monitor the training. For example, you can run this command and check the learning curves at browser.
```bash
--logdir gpt_creditcard_results/ --bind_all 0.0.0.0
```

Once the training converges, we take a checkpoint file which has the smallest validation error.

## Convert checkpoint to nemo file

NeMo library package the checkpoint file and other model artifacts (like model config file and vocab file) into a single `.nemo` file. It makes the process of loading GPT model and running inference easier. 

Let's download the script and convert the checkpoint file into `.nemo` file. Make sure to change the checkpoint file name to the correct name. You can find them in the `gpt_creditcard_results/megatron_gpt/checkpoints/ ` directory.

In [16]:
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/language_modeling/megatron_ckpt_to_nemo.py')

'megatron_ckpt_to_nemo.py'

In [18]:
CHECKPONT_FILE_NAME = megatron_gpt--val_loss=1.23-step=6000-consumed_samples=48000.0-last.ckpt # change it to your checkpoint file name
!torchrun --nproc_per_node=1 megatron_ckpt_to_nemo.py \
    --checkpoint_folder=gpt_creditcard_results/megatron_gpt/checkpoints/ \
    --checkpoint_name={CHECKPONT_FILE_NAME} \
    --nemo_file_path=tabular.nemo \
    --tensor_model_parallel_size={TENSOR_MP_SIZE} \
    --pipeline_model_parallel_size={PIPELINE_MP_SIZE} \
    --gpus_per_node=1 \
    --model_type=gpt

SyntaxError: cannot assign to expression (2274464726.py, line 1)

After running this script, it creates a `tabular.nemo` file which can be used to generate synthetic credit card transactions. 

### Generate synthetic credit card transactions

We come to the last step of this tutorial - generate synthetic credit card transactions!

First let's load the trained model weights, put the NeMo GPT in inference mode and start the text generation server. All of these can be done by running the `megatron_gpt_eval.py` script. 

Let's download the script and configuration file.

In [ ]:
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/language_modeling/megatron_gpt_eval.py')
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/language_modeling/conf/megatron_gpt_inference.yaml', out='conf')

Open another Jupyter notebook or terminal, and run the following in a cell. 
```python
!python megatron_gpt_eval.py \
    gpt_model_file=tabular.nemo \
    prompts=[\'\',\'\'] \
    server=True
```
The text generation server accepts REST API request to send the generated text in the response. Let's use the following Python code to generate some transactions.

In [ ]:
with open(CC_OUTPUT_P, 'rb') as handle:
        cc: ColumnCodes = pickle.load(handle)

token_per_rows = sum(cc.sizes) + 1

batch_size = 4
port_num = 5555
num_of_rows = 10
headers = {"Content-Type": "application/json"}


def request_data(data):
    resp = requests.put('http://localhost:{}/generate'.format(port_num),
                        data=json.dumps(data), headers=headers)
    sentences = resp.json()['sentences']
    return sentences


# generate the initial transactions 
data = {
    "sentences": [""] * batch_size,
    "tokens_to_generate": num_of_rows * token_per_rows,
    "temperature": 1.0,
    "add_BOS": True
}

sentences = request_data(data)

for i in range(batch_size):
    s = sentences[i]
    print(s)

The above code only generate `num_of_rows` of transactions starting from the beginning. If longer sequence is needed, we can run the inference conditioned on the past transactions in a sliding window fashion. For example, first it generates [A, B, C, D, E] transactions conditioned on `<|endoftext|>` token. Then, it conditions on [D, E] and generates [D, E, F, G, H]. Once the long sequence comes to the end indicated by `<|endoftext|>`, it will keep generating new transactions for a different user. For example, after generating [X, Y, Z, <|endoftext|>] in the last run, it will generate [Z, <|endoftext|>, A', B', C'] in the next time, where A', B', C' are transactions for a different user and they are not depending on the other users transaction Z.  

The following code implements the idea above and can be used to generate massive number of transactions in long sequences. The `history_rows` specifies the number of rows of transactions used as conditional context and `num_of_rows` specify the number of rows of transactions to be generated at a time. It saves all the synthetic credit card transactions in the text files whose file names starts with prefix `synthetic`. You can adjust `number_of_blocks`, `batch_size` parameters to generate more transactions.  

In [ ]:
with open(CC_OUTPUT_P, 'rb') as handle:
        cc: ColumnCodes = pickle.load(handle)

batch_size = 4
num_of_rows = 50
token_per_rows = sum(cc.sizes) + 1
history_rows = 40
num_of_blocks = 100

port_num = 5555
headers = {"Content-Type": "application/json"}

prefix_name = 'synthetic'
files = []


for i in range(batch_size):
    files.append(open("{}_{}.txt".format(prefix_name, i), 'w'))


def request_data(data):
    resp = requests.put('http://localhost:{}/generate'.format(port_num),
                        data=json.dumps(data), headers=headers)
    sentences = resp.json()['sentences']
    return sentences


def get_condition_text(sentences, history_rows):
    condition_text = ['\n'.join([ss for ss in s.split(
        '\n')[-(history_rows+1):]]) for s in sentences]
    return condition_text


def get_extra_text(sentences, history_rows):
    extra_text = ['\n'.join([ss for ss in s.split(
        '\n')[history_rows:]]) for s in sentences]
    return extra_text

# generate the initial transactions 
data = {
    "sentences": [""] * batch_size,
    "tokens_to_generate": num_of_rows * token_per_rows,
    "temperature": 1.0,
    "add_BOS": True
}

sentences = request_data(data)

for i in range(batch_size):
    s = sentences[i]
    files[i].write(s.replace('<|endoftext|>', '\n'))

# generate the transactions conditioned on the previous ones
for block in range(num_of_blocks):
    print("block id: {}".format(block))
    condition_text = get_condition_text(sentences, history_rows)
    data = {
        "sentences": condition_text,
        "tokens_to_generate": num_of_rows * token_per_rows,
        "temperature": 1.0,
        "add_BOS": False
    }

    sentences = request_data(data)

    extra_text = get_extra_text(sentences, history_rows)

    for i in range(batch_size):
        s = extra_text[i]
        files[i].write(s.replace('<|endoftext|>', '\n---------------\n'))
        files[i].flush()


for i in range(batch_size):
    files[i].close()

In [ ]:
# check the generated creditcard transaction files
!ls synthetic*.txt

That's it! In this tutorial, you have learned how to train a NeMo GPT model to generate synthetic data. Go ahead and apply it for your own data.